In [ ]:
import fiona
import rasterio, rasterio.mask
import glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with fiona.open("shapefile path", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
print(shapes)

In [ ]:
# Read all tif files in a directory
file_list = [f for f in glob.glob(r"data path\*.tif")]

# create an empty dataframe
df = pd.DataFrame()

for fname in file_list:
    with rasterio.open(fname) as src:
        # subset area of interest
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True) 
        final = np.asarray(out_image, dtype = np.float64, order ='C')
        final[final==-999] = np.nan # remove non value pixels 
        # create a table 
        df[variable_name] = pd.Series(final.ravel()) 

        
        # save the subset data if required
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        with rasterio.open('output_name', "w", **out_meta) as dest:
            dest.write(out_image)

# basic statistics 
df = df.describe()
# export analysis
df.to_excel(r"output_path\excel_file.xlsx") 